In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
# import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
# SYMBOLS = ['WIN$'] # ['WDO$']
# TFRAMES = ['M5','M15','H1','H3','D1']

# Load

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Day Open']    = Calc.groupby(['Symbol','TF','Date'], sort=0)['Open'].transform('first')
    Calc['Day Close']   = Calc.groupby(['Symbol','TF','Date'], sort=0)['Close'].transform('last')
    Calc['ID Chg Bwd']  =                     Calc['Open'] - Calc['Day Open']
    Calc['ID Chg Fwd']  = Calc['Day Close'] - Calc['Open']

    Calc['ID Chg Fwd Abs'] = Calc['ID Chg Fwd'].abs()
    Calc['ID Chg Fwd Pos'] = Calc['ID Chg Fwd'].apply(lambda x: x if (x >= 0) else nan)
    Calc['ID Chg Fwd Neg'] = Calc['ID Chg Fwd'].apply(lambda x: x if (x <  0) else nan)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [27]:
def STATS(Calc, HR):
    pipe = []
    for b in [100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500]:

        Df = Calc[(Calc['Time'] == dt.time(HR,00)) & (Calc['ID Chg Bwd'] > b)]

        pipe.append({
            'Time =': dt.time(HR,00),
            'ID Chg Bwd >': b,
            **Df['ID Chg Fwd Abs'].describe().round(1)
        })
    pass
    return pd.DataFrame(pipe)

STATS(Calc, HR=13)

,Time =,ID Chg Bwd >,count,mean,std,min,25%,50%,75%,max
0,13:00:00,100,574.0,710.3,761.7,0.0,214.2,493.0,939.5,7834.0
1,13:00:00,200,529.0,698.0,691.9,0.0,215.0,495.0,943.0,4886.0
2,13:00:00,500,417.0,737.8,712.2,0.0,234.0,539.0,979.0,4886.0
3,13:00:00,800,325.0,756.0,721.1,0.0,245.0,575.0,1010.0,4886.0
4,13:00:00,1000,279.0,763.3,729.3,0.0,245.0,582.0,1005.5,4886.0
5,13:00:00,1200,231.0,767.1,711.2,5.0,268.5,596.0,1005.5,4886.0
6,13:00:00,1500,151.0,832.6,769.0,5.0,263.5,660.0,1099.5,4886.0
7,13:00:00,1800,94.0,896.9,781.4,5.0,309.2,766.0,1271.8,4886.0
8,13:00:00,2000,69.0,960.7,850.8,5.0,340.0,806.0,1314.0,4886.0
9,13:00:00,2200,52.0,1013.2,903.0,5.0,349.0,831.5,1415.2,4886.0


In [18]:
Calc.groupby('Time')[['ID Chg Fwd']].describe().round(0).astype(int)

ID Chg Fwd                                       
              count mean   std   min   25% 50%   75%   max
Time                                                      
09:00:00       1240  -20  1711 -7641 -1083  65  1016  5768
10:00:00       1240  -16  1597 -6425  -899  19   950  5420
11:00:00       1240  -39  1368 -7315  -698  34   721  5297
12:00:00       1240    2  1189 -6885  -612  18   652  5111
13:00:00       1245  -10  1070 -7834  -502  20   546  5224
14:00:00       1245  -15   953 -7018  -484  17   480  4585
15:00:00       1245  -12   888 -8005  -399   7   453  5697
16:00:00       1245   -2   707 -3739  -312  25   337  5544
17:00:00       1245   -7   477 -4214  -235  11   229  2005
18:00:00        672   20   286 -2530  -130  21   145  1121

In [19]:
Calc.groupby('Time')[['ID Chg Fwd Abs']].describe().round(0).astype(int)

ID Chg Fwd Abs                                       
                  count  mean   std min  25%   50%   75%   max
Time                                                          
09:00:00           1240  1327  1080   0  506  1048  1844  7641
10:00:00           1240  1209  1043   0  423   918  1760  6425
11:00:00           1240  1005   929   0  332   714  1429  7315
12:00:00           1240   859   822   0  285   630  1190  6885
13:00:00           1245   751   761   0  235   522  1010  7834
14:00:00           1245   676   673   0  220   482   923  7018
15:00:00           1245   608   647   0  187   433   800  8005
16:00:00           1245   482   517   0  158   326   639  5544
17:00:00           1245   330   344   0  117   232   421  4214
18:00:00            672   198   207   0   67   140   269  2530

In [20]:
Calc.groupby('Time')[['ID Chg Fwd Pos']].describe().round(0).astype(int)

ID Chg Fwd Pos                                      
                  count  mean   std min  25%  50%   75%   max
Time                                                         
09:00:00            643  1260  1023   0  452  981  1832  5768
10:00:00            633  1169   999   0  429  901  1660  5420
11:00:00            631   949   822   0  314  714  1353  5297
12:00:00            631   846   760   0  299  642  1222  5111
13:00:00            640   721   691   0  233  520   978  5224
14:00:00            628   655   629   0  232  474   901  4585
15:00:00            628   590   594   0  195  446   776  5697
16:00:00            643   465   510   0  158  326   612  5544
17:00:00            639   315   304   0  116  223   414  2005
18:00:00            365   201   193   0   68  134   297  1121

In [21]:
Calc.groupby('Time')[['ID Chg Fwd Neg']].describe().round(0).astype(int)

ID Chg Fwd Neg                                       
                  count  mean   std   min   25%   50%  75% max
Time                                                          
09:00:00            597 -1399  1134 -7641 -1870 -1115 -580  -6
10:00:00            607 -1252  1085 -6425 -1808  -927 -418  -6
11:00:00            609 -1063  1026 -7315 -1547  -713 -355  -6
12:00:00            609  -872   882 -6885 -1168  -617 -258  -7
13:00:00            605  -784   828 -7834 -1046  -530 -237  -5
14:00:00            617  -697   714 -7018  -965  -495 -210  -5
15:00:00            617  -625   698 -8005  -829  -408 -180  -5
16:00:00            602  -501   524 -3739  -669  -329 -158  -5
17:00:00            606  -347   381 -4214  -445  -240 -118  -5
18:00:00            307  -195   223 -2530  -258  -142  -64  -5